In [5]:
import os
from minepy import MINE
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei']

In [6]:
from scipy.spatial.distance import pdist, squareform
import numpy as np

# from numbapro import jit, float32

def distcorr(X, Y):
    """ Compute the distance correlation function

    >>> a = [1,2,3,4,5]
    >>> b = np.array([1,2,9,4,4])
    >>> distcorr(a, b)
    0.762676242417
    """
    X = np.atleast_1d(X)
    Y = np.atleast_1d(Y)
    if np.prod(X.shape) == len(X):
        X = X[:, None]
    if np.prod(Y.shape) == len(Y):
        Y = Y[:, None]
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)
    n = X.shape[0]
    if Y.shape[0] != X.shape[0]:
        raise ValueError('Number of samples must match')
    a = squareform(pdist(X))
    b = squareform(pdist(Y))
    A = a - a.mean(axis=0)[None, :] - a.mean(axis=1)[:, None] + a.mean()
    B = b - b.mean(axis=0)[None, :] - b.mean(axis=1)[:, None] + b.mean()

    dcov2_xy = (A * B).sum()/float(n * n)
    dcov2_xx = (A * A).sum()/float(n * n)
    dcov2_yy = (B * B).sum()/float(n * n)
    dcor = np.sqrt(dcov2_xy)/np.sqrt(np.sqrt(dcov2_xx) * np.sqrt(dcov2_yy))
    return dcor

In [7]:
# index = ["pH", "DO", "COD", "BOD", "AN", "TP"]
# index_id = [4, 3, 2, 1, 0, 5]
index = ["AN", "BOD", "COD", "DO", "pH", "TP", "WT", "CODCR"]

#     columns = ["Mean", "Std", "CV"]

#     yichang_df = pd.DataFrame(index=index, columns=columns)
#     ziya_df = pd.DataFrame(index=index, columns=columns)
name_correct = {"DOX":"DO", "CODMN":"COD", "BOD5":"BOD", "NH3N":"AN", "WT":"WT", 
                "AN":"AN", "BOD":"BOD", "COD":"COD", "PH":"pH", "CODCR":"CODCR",
               "TP": "TP","DO":"DO"}

for file_name in ["yichang", "ziya"]:
    file_path = "../data/" + file_name + ".csv"
    raw_data = pd.read_csv(file_path)

    if file_name == "yichang":
        use_columns = ["NH3N", "BOD5", "CODMN", "DOX", "PH", "TP", "WT"]
        
    else:
        use_columns = ["AN", "BOD", "CODMN", "DO", "PH", "TP", "CODCR"]


    for relation_name in ["MIC", "CS", "DC", "PC", "SC", "KC"]:
        if relation_name 1= ""
        output = pd.DataFrame(columns=["output", 'time_step', "indicator", "cor_value"])
        
        for item_name_2 in use_columns: # item_name_2为目标指标

            for j in range(1, 13):
                i_s_list = []
                i_s_list_2 = []
                for stcd in raw_data['STCD'].unique():
                    stcd_data = raw_data[raw_data['STCD']==stcd]

                    i_s_data = stcd_data.iloc[:, 2:9]

                    i_s_list.append(i_s_data.iloc[:-j, :])
                    i_s_list_2.append(i_s_data.iloc[j:, :])
                time_data_1 = pd.concat(i_s_list, axis=0) # 前
                time_data_2 = pd.concat(i_s_list_2, axis=0) # 后

                time_data_1.index = [n for n in range(len(time_data_1))]
                time_data_2.index = [n for n in range(len(time_data_2))]
                item_name_2_correct = name_correct[item_name_2]
                time_dict = {'output': item_name_2_correct,'time_step': j}

                for item_name in use_columns: # item_name为之前时间的指标

                    data_1 = time_data_1[item_name]   # former
                    data_2 = time_data_2[item_name_2] # latter !!!!
                    
                    df = pd.DataFrame()
                    series_1 = pd.Series(data_1)
                    series_2 = pd.Series(data_2)
                    df['last'] = series_1
                    df['now'] = series_2
                    
                    if relation_name == "MIC":
                        mine = MINE(alpha=0.6, c=15)
                        mine.compute_score(data_1, data_2)                        
                        value = mine.mic()
                    if relation_name == "CS":
                        x = np.array(data_1)
                        y = np.array(data_2)
                        num = x.dot(y.T)
                        denom = np.linalg.norm(x) * np.linalg.norm(y)
                        cosin_cor = num / denom
                        value = cosin_cor
                    if relation_name == "DC":
                        X = np.array(series_1)
                        Y = np.array(series_2)
                        value = distcorr(X, Y)
                    if relation_name == "PC":
                        pearson_cor = df.corr(method='pearson', min_periods=1).iloc[0, 1]
                        value = pearson_cor
                    if relation_name == "SC":
                        spearman_cor = df.corr(method='spearman', min_periods=1).iloc[0, 1]                        
                        value = spearman_cor
                    if relation_name == "KC":
                        kendall_cor = df.corr(method='kendall', min_periods=1).iloc[0, 1]                       
                        value = kendall_cor
                    item_name_correct = name_correct[item_name]
                    time_dict["indicator"] = item_name_correct
                    
                    time_dict["cor_value"] = abs(value)



                    output = output.append(time_dict, ignore_index=True)
        save_name = relation_name + "_indicators_" + file_name + "_7___2.csv"
        save_path = os.path.join("../relation_data", save_name)
        output.to_csv(save_path, encoding='utf-8',index=False)
        print(save_name + " successfully!!!")

PC_indicators_yichang_7___p_value.csv successfully!!!
PC_indicators_ziya_7___p_value.csv successfully!!!


In [9]:
# index = ["pH", "DO", "COD", "BOD", "AN", "TP"]
# index_id = [4, 3, 2, 1, 0, 5]
index = ["AN", "BOD", "COD", "DO", "pH", "TP", "WT", "CODCR"]

#     columns = ["Mean", "Std", "CV"]

#     yichang_df = pd.DataFrame(index=index, columns=columns)
#     ziya_df = pd.DataFrame(index=index, columns=columns)
name_correct = {"DOX":"DO", "CODMN":"COD", "BOD5":"BOD", "NH3N":"AN", "WT":"WT", 
                "AN":"AN", "BOD":"BOD", "COD":"COD", "PH":"pH", "CODCR":"CODCR",
               "TP": "TP","DO":"DO"}

for file_name in ["yichang", "ziya"]:
    file_path = "../data/" + file_name + ".csv"
    raw_data = pd.read_csv(file_path)

    if file_name == "yichang":
        use_columns = ["NH3N", "BOD5", "CODMN", "DOX", "PH", "TP", "WT"]
        
    else:
        use_columns = ["AN", "BOD", "CODMN", "DO", "PH", "TP", "CODCR"]


    for relation_name in ["MIC", "CS", "DC", "PC", "SC", "KC"]:
        if relation_name != "PC":
            continue
        output = pd.DataFrame(columns=["output", 'time_step', "indicator", "cor_value", "p_value"])
        
        for item_name_2 in use_columns: # item_name_2为目标指标

            for j in range(1, 13):
                i_s_list = []
                i_s_list_2 = []
                for stcd in raw_data['STCD'].unique():
                    stcd_data = raw_data[raw_data['STCD']==stcd]

                    i_s_data = stcd_data.iloc[:, 2:9]

                    i_s_list.append(i_s_data.iloc[:-j, :])
                    i_s_list_2.append(i_s_data.iloc[j:, :])
                time_data_1 = pd.concat(i_s_list, axis=0) # 前
                time_data_2 = pd.concat(i_s_list_2, axis=0) # 后

                time_data_1.index = [n for n in range(len(time_data_1))]
                time_data_2.index = [n for n in range(len(time_data_2))]
                item_name_2_correct = name_correct[item_name_2]
                time_dict = {'output': item_name_2_correct,'time_step': j}

                for item_name in use_columns: # item_name为之前时间的指标

                    data_1 = time_data_1[item_name]   # former
                    data_2 = time_data_2[item_name_2] # latter !!!!
                    
                    df = pd.DataFrame()
                    series_1 = pd.Series(data_1)
                    series_2 = pd.Series(data_2)
                    df['last'] = series_1
                    df['now'] = series_2
                    
                    if relation_name == "MIC":
                        mine = MINE(alpha=0.6, c=15)
                        mine.compute_score(data_1, data_2)                        
                        value = mine.mic()
                    if relation_name == "CS":
                        x = np.array(data_1)
                        y = np.array(data_2)
                        num = x.dot(y.T)
                        denom = np.linalg.norm(x) * np.linalg.norm(y)
                        cosin_cor = num / denom
                        value = cosin_cor
                    if relation_name == "DC":
                        X = np.array(series_1)
                        Y = np.array(series_2)
                        value = distcorr(X, Y)
                    if relation_name == "PC":

                        pearson_cor , p_value = stats.pearsonr(df['last'], df['now'])
                        value = pearson_cor
                    if relation_name == "SC":
                        spearman_cor = df.corr(method='spearman', min_periods=1).iloc[0, 1]                        
                        value = spearman_cor
                    if relation_name == "KC":
                        kendall_cor = df.corr(method='kendall', min_periods=1).iloc[0, 1]                       
                        value = kendall_cor
                    item_name_correct = name_correct[item_name]
                    time_dict["indicator"] = item_name_correct
                    
                    time_dict["cor_value"] = abs(value)
                    
                    if p_value < 0.001:
                        p_value = "***"
                    elif p_value < 0.01:
                        p_value = "**"
                    elif p_value < 0.05:
                        p_value = "*"
                    else:
                        p_value = ""
                        count = i + 1
                    time_dict["p_value"] = p_value



                    output = output.append(time_dict, ignore_index=True)
        save_name = relation_name + "_indicators_" + file_name + "_7___p_value.csv"
        save_path = os.path.join("../relation_data", save_name)
        output.to_csv(save_path, encoding='utf-8',index=False)
        print(save_name + " successfully!!!")

PC_indicators_yichang_7___p_value.csv successfully!!!
PC_indicators_ziya_7___p_value.csv successfully!!!
